<a href="https://colab.research.google.com/github/Berke-Yay/Machine-Learning-Models/blob/main/MalariaDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

dataset, info = tfds.load(
    name='malaria',
    split='train',
    as_supervised=True,
    with_info=True,
)

def preprocess(image, label):
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, depth=2)
    return image, label

#Ram'i tek seferde kullanmamak için batch'lere ayırdım
batch_size = 32
preprocessed_dataset = dataset.map(preprocess).batch(batch_size)


#sklearn fonksiyonu yerine biraz daha ilkel bir yaklaşım çalışıyor
split_percentage = 0.8
num_batches = 862
num_train_samples = int(split_percentage * num_batches)
train_data = preprocessed_dataset.take(num_train_samples)
test_data = preprocessed_dataset.skip(num_train_samples)

In [ ]:
class SitmaRecognition(tf.keras.Model):
    def __init__(self):
        super(SitmaRecognition, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.5)
        self.dense2 = tf.keras.layers.Dense(2, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        return self.dense2(x)

model = SitmaRecognition()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


epochs = 1
model.fit(train_data, epochs=epochs)

862/862 [==============================] - 33s 36ms/step - loss: 0.7035 - accuracy: 0.7538


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print("Test Accuracy:", test_accuracy)

173/173 [==============================] - 18s 21ms/step - loss: 0.2440 - accuracy: 0.9172
Test Accuracy: 0.9172413945198059
